In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
from tcompress import register_ as reg
from tcompress import algo_cooling as algo
from tcompress import quf
import time
import numpy as np
from quimb.tensor.belief_propagation.l2bp import L2BP
from quimb.experimental import tnvmc
from tqdm import tqdm

In [2]:
reg.reg_complex_svd()
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)

opt = algo.opt_(progbar=False)
# opt = "auto-hq"

In [3]:
#ITF params and Unitary
J, h = 1, 3.05
dt, depth = 0.05, 60
Lx, Ly = 4, 4
L = Lx * Ly


edges = qtn.edges_2d_square(Lx=Lx, Ly=Ly, cyclic=False)
# --- coordinate maps ---
map_1d_to_2d = {j * Lx + i: (i, j) for j in range(Ly) for i in range(Lx)}
map_2d_to_1d = {(i, j): j * Lx + i for j in range(Ly) for i in range(Lx)}


# --- site list ---
site_tags = [f"I{n}" for n in range(L)]
sites = sorted({(map_2d_to_1d[site],) for edge in edges for site in edge})
L = len(sites)
# --- remap edges to 1D indexing ---
edges_1d = [(map_2d_to_1d[x], map_2d_to_1d[y]) for x, y in edges]



In [4]:
mpo_zz = quf.mpo_z_prod(L, "Z", where_= [L//2, L//2 + 1])
mpo_zz.apply_to_arrays(to_backend)

In [5]:
chi = 60
mpoz2 = qtn.MPO_identity(L) * 1.e-12
for i in range(L):
    for j in range(L):
        if i != j:
            mpo_z = quf.mpo_z_prod(L, "Z", where_= [i, j])
        else:
            mpo_z = qtn.MPO_identity(L)

        mpoz2 += mpo_z
        mpoz2.compress("left", max_bond=chi, cutoff=1.e-12 )

mpoz2  /= L**2
mpoz2.apply_to_arrays(to_backend)
mpoz2.show()

│2│3│3│3│3│3│3│3│3│3│3│3│3│3│2│
●─●─●─●─●─●─●─●─●─●─●─●─●─●─●─●
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │


In [6]:
params = {'rx_depth0': -0.3394656463087295,
 'rzz_depth0': -0.22533130736539966,
 'rx_depth1': -0.430458916463978,
 'rzz_depth1': -0.27541617380526684,
 'rx_depth2': -0.43151334434686744,
 'rzz_depth2': -0.2750848537925995,
 'rx_depth3': -0.33473323423765555,
 'rzz_depth3': -0.2233274482028877}

In [ ]:
p = qtn.MPS_computational_state(["0"]*L)
info_c = {}
p.canonicalize_([L//2], cur_orthog='calc', info=info_c)
p.apply_to_arrays(to_backend)


chi = 120

F_l = []
t_l = []
z2_l = []
zz_l = []
with tqdm(total=depth,  desc="svd:", leave=True, position=0, 
        colour='CYAN', disable = not True) as pbar:

        for depth_ in range(1, depth+1):


                for local_depth in range(4):
                        rx_param = params[f"rx_depth{local_depth}"]
                        rzz_param = params[f"rzz_depth{local_depth}"]
                        rx   = to_backend(qtn.circuit.rx_gate_param_gen([rx_param]))
                        rzz = to_backend(qtn.circuit.rzz_param_gen([rzz_param]))


                        for count, site in enumerate(sites):
                                p.gate_(rx, site, contract=True)

                        for where in edges_1d:
                                p.gate_nonlocal_(rzz, where, max_bond=chi , info=info_c , method='direct', **{"cutoff":1.e-12})
                        
                        for count, site in enumerate(sites):
                                p.gate_(rx, site, contract=True)
            
                
                
                z2 = quf.energy_global(mpoz2, p)
                zz = quf.energy_global(mpo_zz, p)
                
                t_l.append( depth_*0.5  )

                
                z2_l.append( complex(z2).real )
                zz_l.append( complex(zz).real )
                pbar.set_postfix({
                        "t":   f"{round(depth_*0.05,2)}",
                        "norm": complex(p.norm()).real,
                        "<Z2>": complex(z2).real,
                        
                        

                })

                pbar.update(1)



svd::  77%|███████▋  | 46/60 [01:19<00:24,  1.76s/it, t=2.3, norm=0.946, <Z2>=0.292] 

In [ ]:
p.norm()

In [ ]:
p.show()

In [ ]:
# Plot <Z^2> vs time (professional matplotlib line)
import matplotlib.pyplot as plt
# Try a preferred grid style, fall back safely if unavailable
try:
    plt.style.use('seaborn-whitegrid')
except Exception:
    try:
        plt.style.use('seaborn')
    except Exception:
        plt.style.use('ggplot')
# Explicit rc settings to keep appearance consistent
plt.rcParams.update({
    'figure.figsize': (8,4.5),
    'font.size': 12,
    'axes.labelsize': 13,
    'axes.titlesize': 14,
    'lines.linewidth': 2,
    'lines.markersize': 6,
})
# ensure lists exist and are numeric
try:
    assert len(t_l) == len(z2_l) == len(zz_l)
    t_arr = list(map(float, t_l))
    z2_arr = list(map(float, z2_l))
    zz_arr = list(map(float, zz_l))
except Exception as e:
    raise RuntimeError('t_l, z2_l, and zz_l must be defined, same length, and numeric before plotting') from e
fig, ax = plt.subplots()
ax.plot(t_arr, z2_arr, marker='o', linestyle='-', color='C0', label='<Z^2>')
ax.plot(t_arr, zz_arr, marker='o', linestyle='-', color='C1', label='<ZZ>')

ax.set_xlabel('Time (t)')
ax.set_ylabel('<Z^2>')
ax.set_title('Global <Z^2> vs Time')
ax.grid(True, which='both', linestyle='--', linewidth=0.6, alpha=0.7)
ax.legend(frameon=True, fontsize=11)
plt.tight_layout()
plt.show()